In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df["target_up"] = (df["price"].shift(-7) > df["price"]).astype(int)
df = df.iloc[:-7]   # retirer la dernière ligne sans cible

In [4]:
df["target_up"].value_counts(normalize=True)

target_up
0    0.533333
1    0.466667
Name: proportion, dtype: float64

In [5]:
#choix de la cible  

target = df['target_up']
target.shape[0]

330

In [6]:
dataexplicative = df.drop(columns=['date', 'target_up'])

In [7]:
df.shape[0]

330

In [8]:
dataexplicative.shape[0]

330

In [9]:
dataexplicative.tail()

,price,volume,return_1d,return_3d,return_7d,return_14d,MA7,MA30,MA_diff,vol_7d,vol_MA7,highest_30,lowest_30,pos_channel_30
325,90715.753618,5.194012e+10,2.603299,3.394501,-5.491798,-7.830561,89652.380016,93925.278544,-4272.898529,2.948904,6.566432e+10,99618.766232,87737.503329,0.250668
326,91627.763812,5.843824e+10,1.005349,2.526320,-0.915840,-6.495101,89531.391469,93799.859796,-4268.468327,2.702931,6.470270e+10,99618.766232,87737.503329,0.327428
327,88864.312891,6.128648e+10,-3.015954,0.509242,0.423149,-8.322142,89584.883409,93452.284640,-3867.401232,2.393804,5.990474e+10,99618.766232,87737.503329,0.094839
328,87583.285613,5.463719e+10,-1.441554,-3.453058,-3.078082,-7.665657,89187.525993,93051.101953,-3863.575960,2.264400,5.813058e+10,98422.812122,87583.285613,0.000000
329,85762.206093,8.639245e+10,-2.079255,-6.401507,-2.251372,-8.316363,88905.340674,92658.963301,-3753.622627,2.130685,6.254074e+10,98422.812122,85762.206093,0.000000


In [10]:
df.tail()

,date,price,volume,return_1d,return_3d,return_7d,return_14d,MA7,MA30,MA_diff,vol_7d,vol_MA7,highest_30,lowest_30,pos_channel_30,target_up
325,2025-11-10,90715.753618,5.194012e+10,2.603299,3.394501,-5.491798,-7.830561,89652.380016,93925.278544,-4272.898529,2.948904,6.566432e+10,99618.766232,87737.503329,0.250668,0
326,2025-11-11,91627.763812,5.843824e+10,1.005349,2.526320,-0.915840,-6.495101,89531.391469,93799.859796,-4268.468327,2.702931,6.470270e+10,99618.766232,87737.503329,0.327428,0
327,2025-11-12,88864.312891,6.128648e+10,-3.015954,0.509242,0.423149,-8.322142,89584.883409,93452.284640,-3867.401232,2.393804,5.990474e+10,99618.766232,87737.503329,0.094839,0
328,2025-11-13,87583.285613,5.463719e+10,-1.441554,-3.453058,-3.078082,-7.665657,89187.525993,93051.101953,-3863.575960,2.264400,5.813058e+10,98422.812122,87583.285613,0.000000,0
329,2025-11-14,85762.206093,8.639245e+10,-2.079255,-6.401507,-2.251372,-8.316363,88905.340674,92658.963301,-3753.622627,2.130685,6.254074e+10,98422.812122,85762.206093,0.000000,0


In [11]:
target.tail()

325    0
326    0
327    0
328    0
329    0
Name: target_up, dtype: int32

In [12]:

# Variables explicatives et cible
X = dataexplicative
#X = X.drop(columns=['NiveauSiteEUIWN'])
y = target

# Encodage One-Hot basé sur X
X = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)



# Normalisation des données 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # OK : on apprend la normalisation sur l'entraînement
X_test_scaled = scaler.transform(X_test)        # OK : on applique la même normalisation au test

# Appliquer le modèle DummyRegressor
dummy_regressor = DummyClassifier(strategy="stratified")  # 'mean' pour prédire la moyenne des cibles
dummy_regressor.fit(X_train, y_train)

# Faire des prédictions
y_pred = dummy_regressor.predict(X_test)

# Calculer les erreurs
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Vérifier si des valeurs de test sont déjà vues en entraînement
intersection = set(X_train.index).intersection(set(X_test.index))
print(f"🔍 Nombre de valeurs identiques dans Train/Test : {len(intersection)}")

# Accuracy globale
accuracy = accuracy_score(y_test, y_pred)

# Precision par classe
precision_baisse = precision_score(y_test, y_pred, pos_label=0)
precision_hausse = precision_score(y_test, y_pred, pos_label=1)

# Recall par classe
recall_baisse = recall_score(y_test, y_pred, pos_label=0)
recall_hausse = recall_score(y_test, y_pred, pos_label=1)

# F1-score par classe
f1_baisse = f1_score(y_test, y_pred, pos_label=0)
f1_hausse = f1_score(y_test, y_pred, pos_label=1)

print("📊 Accuracy globale :", accuracy)
print("\n🔵 Classe BAISSE (0) :")
print("Precision :", precision_baisse)
print("Recall    :", recall_baisse)
print("F1-score  :", f1_baisse)

print("\n🟢 Classe HAUSSE (1) :")
print("Precision :", precision_hausse)
print("Recall    :", recall_hausse)
print("F1-score  :", f1_hausse)

# Affichage détaillé :
print("\n📄 Classification report :")
print(classification_report(y_test, y_pred, target_names=["baisse","hausse"]))

# Matrice de confusion :
print("🔢 Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))

🔍 Nombre de valeurs identiques dans Train/Test : 0
📊 Accuracy globale : 0.48484848484848486

🔵 Classe BAISSE (0) :
Precision : 0.6551724137931034
Recall    : 0.4418604651162791
F1-score  : 0.5277777777777778

🟢 Classe HAUSSE (1) :
Precision : 0.35135135135135137
Recall    : 0.5652173913043478
F1-score  : 0.43333333333333335

📄 Classification report :
              precision    recall  f1-score   support

      baisse       0.66      0.44      0.53        43
      hausse       0.35      0.57      0.43        23

    accuracy                           0.48        66
   macro avg       0.50      0.50      0.48        66
weighted avg       0.55      0.48      0.49        66

🔢 Matrice de confusion :
[[19 24]
 [10 13]]


In [13]:
# Variables explicatives et cible
X = dataexplicative
#X = X.drop(columns=['NiveauSiteEUIWN'])
y = target


X = pd.get_dummies(X, drop_first=True)  # Conversion des variables catégorielles en variables binaires

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


# 🔹 Normalisation des variables explicatives uniquement
scaler = StandardScaler()

# Appliquer le scaler uniquement aux variables explicatives
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Appliquer la même transformation sur les données de test



logreg = LogisticRegression(
    penalty="elasticnet",
    l1_ratio=0.5,  # mélange L1/L2
    solver="saga"
)
# 🔹 Création du modèle ElasticNet avec alpha fixé à 1
elasticnet = logreg

# Entraînement du modèle
elasticnet.fit(X_train_scaled, y_train)

# 🔹 Faire des prédictions sur l'ensemble de test
y_pred = elasticnet.predict(X_test_scaled)

# 🔹 Évaluer le modèle sur l'ensemble de test
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Affichage des résultats
print(f"📊 R² Score: {r2:.4f}")
print(f"📉 Mean Squared Error: {mse:.4f}")

# Faire des prédictions sur l'ensemble d'entraînement
y_pred_train = elasticnet.predict(X_train_scaled)
# Accuracy globale
accuracy = accuracy_score(y_test, y_pred)

# Precision par classe
precision_baisse = precision_score(y_test, y_pred, pos_label=0)
precision_hausse = precision_score(y_test, y_pred, pos_label=1)

# Recall par classe
recall_baisse = recall_score(y_test, y_pred, pos_label=0)
recall_hausse = recall_score(y_test, y_pred, pos_label=1)

# F1-score par classe
f1_baisse = f1_score(y_test, y_pred, pos_label=0)
f1_hausse = f1_score(y_test, y_pred, pos_label=1)

print("📊 Accuracy globale :", accuracy)
print("\n🔵 Classe BAISSE (0) :")
print("Precision :", precision_baisse)
print("Recall    :", recall_baisse)
print("F1-score  :", f1_baisse)

print("\n🟢 Classe HAUSSE (1) :")
print("Precision :", precision_hausse)
print("Recall    :", recall_hausse)
print("F1-score  :", f1_hausse)

# Affichage détaillé :
print("\n📄 Classification report :")
print(classification_report(y_test, y_pred, target_names=["baisse","hausse"]))



📊 R² Score: -0.6016
📉 Mean Squared Error: 0.3636
📊 Accuracy globale : 0.6363636363636364

🔵 Classe BAISSE (0) :
Precision : 0.7714285714285715
Recall    : 0.627906976744186
F1-score  : 0.6923076923076923

🟢 Classe HAUSSE (1) :
Precision : 0.4838709677419355
Recall    : 0.6521739130434783
F1-score  : 0.5555555555555556

📄 Classification report :
              precision    recall  f1-score   support

      baisse       0.77      0.63      0.69        43
      hausse       0.48      0.65      0.56        23

    accuracy                           0.64        66
   macro avg       0.63      0.64      0.62        66
weighted avg       0.67      0.64      0.64        66



In [14]:

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split



# Variables explicatives et cible
X = dataexplicative
y = target



# 🎲 Séparation des données avec stratification basée sur les quartiles de la cible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,  random_state=42#, stratify = y_quartiles  # Stratification sur y_quartiles
)

# 🔧 Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude='object').columns.tolist()),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), X_train.select_dtypes(include='object').columns.tolist())
    ]
)

# 💡 Pipeline complet avec GridSearchCV
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', LogisticRegression(
        penalty='elasticnet',
        solver='saga',
        max_iter=1000,
        random_state=42
    ))
])

param_grid = {
    'model__C': [0.01, 0.1, 1, 10],
    'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 🚀 Entraînement avec GridSearchCV
grid_search.fit(X_train, y_train)

# ✅ Meilleur modèle trouvé
best_model = grid_search.best_estimator_

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur F1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

# 📊 Évaluation globale
accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("📊 Évaluation sur TEST")
print(f"🎯 Accuracy        : {accuracy:.4f}")
print(f"💠 F1 Macro        : {f1_macro:.4f}")
print(f"🔷 F1 Weighted     : {f1_weighted:.4f}")

# 📊 Rapport détaillé par classe
print("\n🔍 Rapport détaillé par classe :")
print(classification_report(y_test, y_pred, target_names=['BAISSE', 'HAUSSE']))

# 🧩 Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("\n🧩 Matrice de confusion :")
print(cm)

# 🔵 Évaluation entraînement (pour voir l’overfitting)
y_pred_train = best_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
f1_macro_train = f1_score(y_train, y_pred_train, average='macro')

print("\n📊 Évaluation sur TRAIN")
print(f"🎯 Accuracy (train) : {accuracy_train:.4f}")
print(f"💠 F1 Macro (train) : {f1_macro_train:.4f}")

# 📈 Extraire les coefficients du modèle
coefs = best_model.named_steps['model'].coef_.flatten()
features = best_model.named_steps['preprocessing'].get_feature_names_out()

coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient': coefs
}).sort_values(by='Coefficient', ascending=False)

print("\n📈 Coefficients du modèle :")
print(coefficients)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
✅ Meilleur alpha trouvé : {'model__C': 0.1, 'model__l1_ratio': 0.1}
📊 Meilleur F1_macro (validation croisée) : 0.6427
📊 Évaluation sur TEST
🎯 Accuracy        : 0.6667
💠 F1 Macro        : 0.6510
🔷 F1 Weighted     : 0.6734

🔍 Rapport détaillé par classe :
              precision    recall  f1-score   support

      BAISSE       0.78      0.67      0.72        43
      HAUSSE       0.52      0.65      0.58        23

    accuracy                           0.67        66
   macro avg       0.65      0.66      0.65        66
weighted avg       0.69      0.67      0.67        66


🧩 Matrice de confusion :
[[29 14]
 [ 8 15]]

📊 Évaluation sur TRAIN
🎯 Accuracy (train) : 0.6780
💠 F1 Macro (train) : 0.6767

📈 Coefficients du modèle :
                Feature  Coefficient
2        num__return_1d     0.031990
3        num__return_3d     0.004541
13  num__pos_channel_30     0.001362
1           num__volume     0.000000
6              num_

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder  # Nous utilisons category_encoders pour le Target Encoding

# Variables explicatives et cible
X = dataexplicative
y = target



# 🎲 Séparation des données avec stratification basée sur les quartiles de la cible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42#, stratify=y_quartiles  # Stratification sur y_quartiles
)

# 🔧 Pipeline de prétraitement
# Nous remplaçons le OneHotEncoder par un TargetEncoder pour les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude='object').columns.tolist()),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), X_train.select_dtypes(include='object').columns.tolist())
    ]
)

# 💡 Pipeline complet avec GridSearchCV
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(random_state=42))  # Modèle ElasticNet
])

# Paramètres pour GridSearchCV
param_grid = {
    'model__max_depth': [10, 5, 20, 30],  # Using model__ to specify RandomForestRegressor
    'model__n_estimators': [10, 50, 100, 200],  # Add more parameters for the model if needed
    'model__min_samples_split': [3, 2, 4, 5, 6]
    # Add other hyperparameters for preprocessing steps if needed
}

# GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 🚀 Entraînement avec GridSearchCV
grid_search.fit(X_train, y_train)

# ✅ Meilleur modèle trouvé
best_model = grid_search.best_estimator_

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur f1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("📊 Évaluation sur TEST")
print(f"🎯 Accuracy        : {accuracy:.4f}")
print(f"💠 F1 Macro        : {f1_macro:.4f}")
print(f"🔷 F1 Weighted     : {f1_weighted:.4f}")

# 📊 Rapport détaillé par classe
print("\n🔍 Rapport détaillé par classe :")
print(classification_report(y_test, y_pred, target_names=['BAISSE', 'HAUSSE']))

# 🧩 Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("\n🧩 Matrice de confusion :")
print(cm)

# 🔵 Évaluation entraînement (pour voir l’overfitting)
y_pred_train = best_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
f1_macro_train = f1_score(y_train, y_pred_train, average='macro')

print("\n📊 Évaluation sur TRAIN")
print(f"🎯 Accuracy (train) : {accuracy_train:.4f}")
print(f"💠 F1 Macro (train) : {f1_macro_train:.4f}")

# 🔵 Prédictions TEST
y_pred_test = best_model.predict(X_test)

# 📊 Évaluation TEST
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_macro_test = f1_score(y_test, y_pred_test, average='macro')

print("\n📊 Évaluation sur TEST")
print(f"🎯 Accuracy (test) : {accuracy_test:.4f}")
print(f"💠 F1 Macro (test) : {f1_macro_test:.4f}")


Fitting 5 folds for each of 80 candidates, totalling 400 fits
✅ Meilleur alpha trouvé : {'model__max_depth': 20, 'model__min_samples_split': 2, 'model__n_estimators': 50}
📊 Meilleur f1_macro (validation croisée) : 0.7712
📊 Évaluation sur TEST
🎯 Accuracy        : 0.8939
💠 F1 Macro        : 0.8864
🔷 F1 Weighted     : 0.8953

🔍 Rapport détaillé par classe :
              precision    recall  f1-score   support

      BAISSE       0.95      0.88      0.92        43
      HAUSSE       0.81      0.91      0.86        23

    accuracy                           0.89        66
   macro avg       0.88      0.90      0.89        66
weighted avg       0.90      0.89      0.90        66


🧩 Matrice de confusion :
[[38  5]
 [ 2 21]]

📊 Évaluation sur TRAIN
🎯 Accuracy (train) : 1.0000
💠 F1 Macro (train) : 1.0000

📊 Évaluation sur TEST
🎯 Accuracy (test) : 0.8939
💠 F1 Macro (test) : 0.8864


In [16]:
#decisiontree
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder  # Nous utilisons category_encoders pour le Target Encoding

# Variables explicatives et cible
X = dataexplicative
y = target


# 🎲 Séparation des données avec stratification basée sur les quartiles de la cible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42#, stratify=y_quartiles  # Stratification sur y_quartiles
)

# 🔧 Pipeline de prétraitement
# Nous remplaçons le OneHotEncoder par un TargetEncoder pour les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude='object').columns.tolist()),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), X_train.select_dtypes(include='object').columns.tolist())
    ]
)

# 💡 Pipeline complet avec GridSearchCV
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', DecisionTreeClassifier(random_state=42))  # Modèle ElasticNet
])


param_grid = {
    'model__max_depth': [10, 5, 20, 30],  # Using model__ to specify RandomForestRegressor
    'model__min_samples_leaf': [2, 3, 4, 5, 6],  # Add more parameters for the model if needed
    'model__min_samples_split': [3, 2, 4]
    # Add other hyperparameters for preprocessing steps if needed
}

# GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 🚀 Entraînement avec GridSearchCV
grid_search.fit(X_train, y_train)

# ✅ Meilleur modèle trouvé
best_model = grid_search.best_estimator_

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur f1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur f1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("📊 Évaluation sur TEST")
print(f"🎯 Accuracy        : {accuracy:.4f}")
print(f"💠 F1 Macro        : {f1_macro:.4f}")
print(f"🔷 F1 Weighted     : {f1_weighted:.4f}")

# 📊 Rapport détaillé par classe
print("\n🔍 Rapport détaillé par classe :")
print(classification_report(y_test, y_pred, target_names=['BAISSE', 'HAUSSE']))

# 🧩 Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("\n🧩 Matrice de confusion :")
print(cm)

# 🔵 Évaluation entraînement (pour voir l’overfitting)
y_pred_train = best_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
f1_macro_train = f1_score(y_train, y_pred_train, average='macro')

print("\n📊 Évaluation sur TRAIN")
print(f"🎯 Accuracy (train) : {accuracy_train:.4f}")
print(f"💠 F1 Macro (train) : {f1_macro_train:.4f}")


Fitting 5 folds for each of 60 candidates, totalling 300 fits
✅ Meilleur alpha trouvé : {'model__max_depth': 10, 'model__min_samples_leaf': 5, 'model__min_samples_split': 3}
📊 Meilleur f1_macro (validation croisée) : 0.7114
✅ Meilleur alpha trouvé : {'model__max_depth': 10, 'model__min_samples_leaf': 5, 'model__min_samples_split': 3}
📊 Meilleur f1_macro (validation croisée) : 0.7114
📊 Évaluation sur TEST
🎯 Accuracy        : 0.8333
💠 F1 Macro        : 0.8215
🔷 F1 Weighted     : 0.8354

🔍 Rapport détaillé par classe :
              precision    recall  f1-score   support

      BAISSE       0.90      0.84      0.87        43
      HAUSSE       0.73      0.83      0.78        23

    accuracy                           0.83        66
   macro avg       0.82      0.83      0.82        66
weighted avg       0.84      0.83      0.84        66


🧩 Matrice de confusion :
[[36  7]
 [ 4 19]]

📊 Évaluation sur TRAIN
🎯 Accuracy (train) : 0.9015
💠 F1 Macro (train) : 0.9015


In [17]:
#gradientboosting
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder  # Nous utilisons category_encoders pour le Target Encoding

# Variables explicatives et cible
X = dataexplicative
y = target



# 🎲 Séparation des données avec stratification basée sur les quartiles de la cible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42#, stratify=y_quartiles  # Stratification sur y_quartiles
)

# 🔧 Pipeline de prétraitement
# Nous remplaçons le OneHotEncoder par un TargetEncoder pour les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude='object').columns.tolist()),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), X_train.select_dtypes(include='object').columns.tolist())
    ]
)

# 💡 Pipeline complet avec GridSearchCV
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', GradientBoostingClassifier(random_state=42))  # Modèle ElasticNet
])

param_grid = {
    'model__max_depth': [10, 5, 2, 3],  # Using model__ to specify RandomForestRegressor
    'model__n_estimators': [100, 200, 300, 50],  # Add more parameters for the model if needed
    'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2, 0.3],
    # Add other hyperparameters for preprocessing steps if needed
}

# GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 🚀 Entraînement avec GridSearchCV
grid_search.fit(X_train, y_train)

# ✅ Meilleur modèle trouvé
best_model = grid_search.best_estimator_

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur f1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("📊 Évaluation sur TEST")
print(f"🎯 Accuracy        : {accuracy:.4f}")
print(f"💠 F1 Macro        : {f1_macro:.4f}")
print(f"🔷 F1 Weighted     : {f1_weighted:.4f}")

# 📊 Rapport détaillé par classe
print("\n🔍 Rapport détaillé par classe :")
print(classification_report(y_test, y_pred, target_names=['BAISSE', 'HAUSSE']))

# 🧩 Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("\n🧩 Matrice de confusion :")
print(cm)

# 🔵 Évaluation entraînement (pour voir l’overfitting)
y_pred_train = best_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
f1_macro_train = f1_score(y_train, y_pred_train, average='macro')

print("\n📊 Évaluation sur TRAIN")
print(f"🎯 Accuracy (train) : {accuracy_train:.4f}")
print(f"💠 F1 Macro (train) : {f1_macro_train:.4f}")


Fitting 5 folds for each of 96 candidates, totalling 480 fits
✅ Meilleur alpha trouvé : {'model__learning_rate': 0.2, 'model__max_depth': 5, 'model__n_estimators': 300}
📊 Meilleur f1_macro (validation croisée) : 0.7862
📊 Évaluation sur TEST
🎯 Accuracy        : 0.8636
💠 F1 Macro        : 0.8581
🔷 F1 Weighted     : 0.8666

🔍 Rapport détaillé par classe :
              precision    recall  f1-score   support

      BAISSE       0.97      0.81      0.89        43
      HAUSSE       0.73      0.96      0.83        23

    accuracy                           0.86        66
   macro avg       0.85      0.89      0.86        66
weighted avg       0.89      0.86      0.87        66


🧩 Matrice de confusion :
[[35  8]
 [ 1 22]]

📊 Évaluation sur TRAIN
🎯 Accuracy (train) : 1.0000
💠 F1 Macro (train) : 1.0000
